<a href="https://colab.research.google.com/github/shio0418/googleSTEP/blob/main/STEP_hashmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 課題１
#### 現在の状況
Functional testは通ったが、item_countによりハッシュテーブルのサイズを可変にするような関数を実装したところ動かなくなった。。。

再ハッシュするにあたり、selfを新しく作ったハッシュテーブルに書き換えることで対応しようとしていたことが多分原因です、、

selfが動いているか確認し、いい感じならnew_hashを全てselfのやつにコピーすればいいかも



In [ ]:
import random, sys, time

###########################################################################
#                                                                         #
# Implement a hash table from scratch! (⑅•ᴗ•⑅)                            #
#                                                                         #
# Please do not use Python's dictionary or Python's collections library.  #
# The goal is to implement the data structure yourself.                   #
#                                                                         #
###########################################################################

# Hash function.
#
# |key|: string
# Return value: a hash value
def calculate_hash(key):
    assert type(key) == str
    # Note: This is not a good hash function. Do you see why?
    hash = 0
    for i in key:
        hash += ord(i)
    return hash


# An item object that represents one key - value pair in the hash table.
class Item:
    # |key|: The key of the item. The key must be a string.
    # |value|: The value of the item.
    # |next|: The next item in the linked list. If this is the last item in the
    #         linked list, |next| is None.
    def __init__(self, key, value, next):
        assert type(key) == str
        self.key = key
        self.value = value
        self.next = next # 次のノード


# The main data structure of the hash table that stores key - value pairs.
# The key must be a string. The value can be any type.
#
# |self.bucket_size|: The bucket size.
# |self.buckets|: An array of the buckets. self.buckets[hash % self.bucket_size]
#                 stores a linked list of items whose hash value is |hash|.
# |self.item_count|: The total number of items in the hash table.


In [ ]:
class HashTable:
    prime_list = [97] # 素数の値を保持するリスト

    # Initialize the hash table.
    def __init__(self):
        # Set the initial bucket size to 97. A prime number is chosen to reduce
        # hash conflicts.
        self.bucket_size = 97
        self.buckets = [None] * self.bucket_size
        self.item_count = 0

    def make_bigger_prime(self,num): # num以上でnumに最も近い素数を生成する関数
        near_prime = -100
        for prime in self.prime_list:
          if prime >= num & prime < near_prime:
            near_prime = prime
        if near_prime > 0:
          return near_prime

        while True:
          divide = 2
          while True:
            if num % divide == 0:
              break
            elif divide == num // 2:
              self.prime_list.append(num)
              return num
            else:
              if divide % 2 == 0:
                divide += 1
              else:
                divide += 2
          num += 2

    def make_hash_bigger(self): # HashMapを大きくする関数
        new_hashtable = HashTable() # 新しいhashtable
        new_hashtable.bucket_size = self.make_bigger_prime(2*self.bucket_size+1)
        current_index = 0

        while current_index < self.bucket_size:
          current_item = self.buckets[current_index]
          while current_item:
            new_hashtable.put(current_item.key,current_item.value)
            current_item = current_item.next
          current_index += 1
        return new_hashtable


    def make_smaller_prime(self,num): # num以上でnumに最も近い素数を生成する関数
        near_prime = num
        for prime in self.prime_list:
          if prime <= num & prime > near_prime:
            near_prime = prime
        if near_prime < num:
          return near_prime

        while True:
          divide = 2
          while True:
            if num % divide == 0:
              break
            elif divide == num // 2:
              self.prime_list.append(num)
              return num
            else:
              if divide % 2 == 0:
                divide += 1
              else:
                divide += 2

          num -= 2
          if (num < 0):
            return 1

    def make_hash_smaller(self): # HashMapを大きくする関数
        new_hashtable = HashTable() # 新しいhashtable
        new_hashtable.bucket_size = self.make_smaller_prime(self.bucket_size // 2)
        current_index = 0

        while current_index < self.bucket_size:
          current_item = self.buckets[current_index]
          while current_item:
            new_hashtable.put(current_item.key,current_item.value)
            current_item = current_item.next
          current_index += 1
        return new_hashtable


    # Put an item to the hash table. If the key already exists, the
    # corresponding value is updated to a new value.
    #
    # |key|: The key of the item.
    # |value|: The value of the item.
    # Return value: True if a new item is added. False if the key already exists
    #               and the value is updated.すでにkeyが存在していたらfalse
    def put(self, key, value):
        assert type(key) == str
        self.check_size() # Note: Don't remove this code.
        bucket_index = calculate_hash(key) % self.bucket_size
        item = self.buckets[bucket_index]

        if self.item_count >= self.bucket_size * 0.7: # 要素数がテーブルサイズの70%を上回ったら2倍に拡張
          self = self.make_hash_bigger()
        while item:
            if item.key == key:
                item.value = value
                return False
            item = item.next
        new_item = Item(key, value, self.buckets[bucket_index])
        self.buckets[bucket_index] = new_item
        self.item_count += 1
        return True

    # Get an item from the hash table.
    #
    # |key|: The key.
    # Return value: If the item is found, (the value of the item, True) is
    #               returned. Otherwise, (None, False) is returned.
    def get(self, key):
        assert type(key) == str
        self.check_size() # Note: Don't remove this code.
        bucket_index = calculate_hash(key) % self.bucket_size
        item = self.buckets[bucket_index]

        while item:
            if item.key == key:
                return (item.value, True)
            item = item.next
        return (None, False)

    # Delete an item from the hash table.
    #
    # |key|: The key.
    # Return value: True if the item is found and deleted successfully. False
    #               otherwise.
    def delete(self, key):
        assert type(key) == str
        #self.check_size(). /*これは必要？*/
        if self.item_count <= self.bucket_size * 0.3:
          self = self.make_hash_smaller()
        bucket_index = calculate_hash(key) % self.bucket_size
        item = self.buckets[bucket_index]
        pre_item = None # 先頭処理の時に困るので

        while item:
          if item.key == key:
            if pre_item == None:
              self.buckets[bucket_index] = item.next # 先頭をitemの次の要素に
            else:
              pre_item.next = item.next
            self.item_count -= 1
            return True
          pre_item = item
          item = item.next
        return False

    # Return the total number of items in the hash table.
    def size(self):
        return self.item_count

    # Check that the hash table has a "reasonable" bucket size.
    # The bucket size is judged "reasonable" if it is smaller than 100 or
    # the buckets are 30% or more used.
    #
    # Note: Don't change this function.
    def check_size(self):
        assert (self.bucket_size < 100 or
                self.item_count >= self.bucket_size * 0.3)



In [ ]:
# Test the functional behavior of the hash table.
def functional_test():
    hash_table = HashTable()

    assert hash_table.put("aaa", 1) == True
    assert hash_table.get("aaa") == (1, True)
    assert hash_table.size() == 1

    assert hash_table.put("bbb", 2) == True
    assert hash_table.put("ccc", 3) == True
    assert hash_table.put("ddd", 4) == True
    assert hash_table.get("aaa") == (1, True)
    assert hash_table.get("bbb") == (2, True)
    assert hash_table.get("ccc") == (3, True)
    assert hash_table.get("ddd") == (4, True)
    assert hash_table.get("a") == (None, False)
    assert hash_table.get("aa") == (None, False)
    assert hash_table.get("aaaa") == (None, False)
    assert hash_table.size() == 4

    assert hash_table.put("aaa", 11) == False
    assert hash_table.get("aaa") == (11, True)
    assert hash_table.size() == 4

    assert hash_table.delete("aaa") == True
    assert hash_table.get("aaa") == (None, False)
    assert hash_table.size() == 3

    assert hash_table.delete("a") == False
    assert hash_table.delete("aa") == False
    assert hash_table.delete("aaa") == False
    assert hash_table.delete("aaaa") == False

    assert hash_table.delete("ddd") == True
    assert hash_table.delete("ccc") == True
    assert hash_table.delete("bbb") == True
    assert hash_table.get("aaa") == (None, False)
    assert hash_table.get("bbb") == (None, False)
    assert hash_table.get("ccc") == (None, False)
    assert hash_table.get("ddd") == (None, False)
    assert hash_table.size() == 0

    assert hash_table.put("abc", 1) == True
    assert hash_table.put("acb", 2) == True
    assert hash_table.put("bac", 3) == True
    assert hash_table.put("bca", 4) == True
    assert hash_table.put("cab", 5) == True
    assert hash_table.put("cba", 6) == True
    assert hash_table.get("abc") == (1, True)
    assert hash_table.get("acb") == (2, True)
    assert hash_table.get("bac") == (3, True)
    assert hash_table.get("bca") == (4, True)
    assert hash_table.get("cab") == (5, True)
    assert hash_table.get("cba") == (6, True)
    assert hash_table.size() == 6

    assert hash_table.delete("abc") == True
    assert hash_table.delete("cba") == True
    assert hash_table.delete("bac") == True
    assert hash_table.delete("bca") == True
    assert hash_table.delete("acb") == True
    assert hash_table.delete("cab") == True
    assert hash_table.size() == 0
    print("Functional tests passed!")



In [ ]:
# Test the performance of the hash table.
#
# Your goal is to make the hash table work with mostly O(1).
# If the hash table works with mostly O(1), the execution time of each iteration
# should not depend on the number of items in the hash table. To achieve the
# goal, you will need to 1) implement rehashing (Hint: expand / shrink the hash
# table when the number of items in the hash table hits some threshold) and
# 2) tweak the hash function (Hint: think about ways to reduce hash conflicts).
def performance_test():
    hash_table = HashTable()

    for iteration in range(100):
        begin = time.time()
        random.seed(iteration)
        for i in range(10000):
            rand = random.randint(0, 100000000)
            hash_table.put(str(rand), str(rand))
        random.seed(iteration)
        for i in range(10000):
            rand = random.randint(0, 100000000)
            hash_table.get(str(rand))
        end = time.time()
        print("%d %.6f" % (iteration, end - begin))

    for iteration in range(100):
        random.seed(iteration)
        for i in range(10000):
            rand = random.randint(0, 100000000)
            hash_table.delete(str(rand))

    assert hash_table.size() == 0
    print("Performance tests passed!")


if __name__ == "__main__":
    functional_test()
    performance_test()

(11, True)


AssertionError: 

In [ ]:
test_hash = HashTable()
prime = test_hash.make_prime(99)
print(prime)

AttributeError: 'HashTable' object has no attribute 'make_prime'